In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

fontSize=12
plt.rcParams.update({
'font.size': fontSize,         # Set font size for labels, legends, and ticks
'axes.labelsize': fontSize,    # X and Y labels
'legend.fontsize': fontSize,   # Legend
'xtick.labelsize': fontSize,   # X-axis tick labels
'ytick.labelsize': fontSize    # Y-axis tick labels
})


def read_data_npy(loc):
  file = np.load(loc, allow_pickle=True).item()
  n_qubits = list(file.keys())
  df = pd.DataFrame(columns=['n', 'm', 'time'])
  for n in n_qubits:
    if n != 11:

      data = file[n]
      # Convert to long-form DataFrame
      temp = pd.DataFrame([(n, k, val) for k, arr in data.items() for val in arr], columns=["n", "m", "time"])
      df = pd.concat([df, temp])

  return df


def mean_df(df):
  mean_df = df.groupby(["m", 'n'])["time"].mean().reset_index()

  return mean_df

def mean_plot(df):
  m = df['m'].unique()

  for val in df['n'].unique():
    subset = mean_df[mean_df['n'] == val]
    # plt.loglog(2**subset['n'], subset['time'], marker='o', label=f'm = {val}')
    plt.plot(m, subset['time'], 'o--', label=f'n = {val}')

  # plt.xscale('log')
  # plt.yscale('log')
  plt.legend()
  plt.grid(True, which="both", ls='--')
  plt.xlabel(r'$M$')
  plt.ylabel('Time (s)')
  plt.xticks([1000, 2000, 3000, 4000, 5000])
  # plt.show()


def error_bar(df):
  # Group by 'key' and compute mean and std
  agg_df = df.groupby("key")["value"].agg(["mean", "std"]).reset_index()

  # Plot: keys on x-axis, values on y-axis with error bars
  plt.errorbar(
      agg_df["key"],       # x-axis: keys
      agg_df["mean"],      # y-axis: means
      yerr=agg_df["std"],  # y-error: std deviations
      fmt='o',            # circle markers with line
      capsize=5,           # caps on error bars
      label=str(n)
  )


def plot_loglog_K(mean_df, m_value):
  subset = mean_df[mean_df['m'] == m_value]
  # Compute d = 2^n
  subset['d'] = 2 ** subset['n']

  # Plot measured time
  plt.loglog(subset['d'], subset['time'], 'o-', label= fr'observed runtime $(M = 10^3)$', markersize=8)

  # Plot theoretical O(d) and O(d²) for reference
  d_values = np.linspace(subset['d'].min(), subset['d'].max(), 100)
  C1 = subset['time'].min() / d_values.min()  # simple scaling for visibility
  C2 = subset['time'].min() / (d_values.min() ** 2)
  C3 = subset['time'].min() / (d_values.min()*np.log2(d_values.min()))

  plt.loglog(d_values, C1 * d_values, '--', label=r'$O(d)$')
  plt.loglog(d_values, C2 * d_values**2, '--', label=r'$O(d^2)$')
  plt.loglog(d_values, C3 * d_values*np.log2(d_values), '--', label=r'$O(d \log d)$')

  # Labels and legend
  plt.xlabel(r'd = $2^n$')
  plt.ylabel('Time (s)')
  plt.legend()
  plt.grid(True, which="both", ls='--')
  plt.tight_layout()

data = read_data_npy('/content/measTimeLowMem_m_1K_2K_3K_4K_5K.npy')
data = data[data['n'].isin([ 15, 16, 17, 18])]
data = data[data['m'].isin([1000, 2000, 3000, 4000, 5000])]
mean_df = mean_df(data)
# plot_loglog_K(mean_df, 1000)
mean_plot(mean_df)
plt.savefig('runtimeMeanLinear2.pdf', format='pdf')
plt.show()


